In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

%matplotlib inline

# Gets the GPU if there is one, otherwise the cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Concentrating on the first 100 samples
n_samples = 100

X_train = datasets.MNIST(root='./kaggle/input', train=True, download=True,
                         transform=None)
type(X_train)

In [ ]:
# Leaving only labels 0 and 1 
idx = np.append(np.where(X_train.targets == 0)[0][:n_samples], 
                np.where(X_train.targets == 1)[0][:n_samples])

X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]
print("type(X_train.data): ", type(X_train.data))

print("X_train.data.shape: ", X_train.data.shape)
print("X_train.targets.shape: ", X_train.targets.shape)

X_train_targets_numpy = X_train.targets.numpy()
print("type(X_train_targets_numpy), X_train_targets_numpy.shape: ", type(X_train_targets_numpy), X_train_targets_numpy.shape)

print("X_train_targets_numpy: ", X_train_targets_numpy)

X_train_numpy = X_train.data.numpy()

print("type(X_train_numpy): ", type(X_train_numpy))
print("X_train_numpy.shape: ", X_train_numpy.shape)

X_train_numpy = X_train_numpy.reshape((X_train_numpy.shape[0], -1), order='F')
X_train_numpy = np.int64(X_train_numpy)

print("X_train_numpy.shape: ", X_train_numpy.shape)

train_set = pd.DataFrame(X_train_numpy)
train_set['label'] = X_train_targets_numpy
print("no. of train columns: ", len(train_set.columns))

print("train_set.dtypes: ", train_set.dtypes)


In [ ]:
# shift column 'C' to first position 
first_column = train_set.pop('label') 
  
# insert column using insert(position,column_name,first_column) function 
train_set.insert(0, 'label', first_column)

In [ ]:
train_set.sample(5)

In [ ]:
n_samples = 50

X_test = datasets.MNIST(root='./kaggle/input', train=False, download=True,
                        transform=transforms.Compose([transforms.ToTensor()]))

print(type(X_test))

idx = np.append(np.where(X_test.targets == 0)[0][:n_samples], 
                np.where(X_test.targets == 1)[0][:n_samples])

X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]

X_test_targets_numpy = X_test.targets.numpy()
print("type(X_test_targets_numpy), X_test_targets_numpy.shape: ", type(X_test_targets_numpy), X_test_targets_numpy.shape)

X_test_numpy = X_test.data.numpy()
print("type(X_test_numpy), X_test_numpy.shape: ", type(X_test_numpy), X_test_numpy.shape)


X_test_numpy = X_test_numpy.reshape((X_test_numpy.shape[0], -1), order='F')
X_test_numpy = np.int64(X_test_numpy)
print("X_test_numpy.shape: ", X_test_numpy.shape)

test_images = pd.DataFrame(X_test_numpy)
test_images['label'] = X_test_targets_numpy
print("no. of test columns: ", len(test_images.columns))


In [ ]:
# shift column 'C' to first position 
first_column = test_images.pop('label') 
  
# insert column using insert(position,column_name,first_column) function 
test_images.insert(0, 'label', first_column)

In [ ]:
test_images.sample(5)

In [ ]:
train_set.dtypes.unique(), test_images.dtypes.unique()

In [ ]:
#train_set_csv = pd.read_csv("../input/mnist-in-csv/mnist_train.csv")
#train_set_csv.dtypes.unique()

In [ ]:
#train_set_csv.head()

In [ ]:
#train_set_csv.label.unique()

In [ ]:
#test_images_csv = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")
#test_images_csv.dtypes.unique()

In [ ]:
#train_set = train_set_csv.loc[(train_set_csv["label"] == 0) | (train_set_csv["label"] == 1)]
#train_set = train_set.head(200)
#train_set.label.unique()

In [ ]:
#test_images = test_images_csv.loc[(test_images_csv["label"] == 0) | (test_images_csv["label"] == 1)]
#test_images = test_images.head(100)
#test_images.label.unique()

In [ ]:
test_images.label.value_counts()

In [ ]:
train_set.label.value_counts()

In [ ]:
# Quick function that gets how many out of 'preds' match 'labels'
# To be used much later
def get_num_correct(preds, labels):
    #print(preds, labels)
    #print(preds.argmax(dim=1).eq(labels).sum().item())
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
# Split the train set so there is also a validation set
train_images, val_images, train_labels, val_labels = train_test_split(train_set.iloc[:, 1:], 
                                                                     train_set.iloc[:, 0], 
                                                                     test_size=0.2, random_state=23)

# Reset indices so the Dataset can find them with __getitem__ easily
train_images.reset_index(drop=True, inplace=True)
val_images.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

In [ ]:
len(train_images), len(val_images)

In [ ]:
type(train_labels), train_labels.unique()

In [ ]:
type(val_labels), val_labels.unique()

In [ ]:
pd.set_option('display.max_columns', 28)
val_images[-3:]

In [ ]:
train_images[:5]

In [ ]:
train_images.shape, type(train_images), val_images.shape

In [ ]:
train_labels.value_counts()

In [ ]:
val_labels.value_counts()

In [ ]:
# Let's just display a number as a dataframe for fun 
# (I think it looks cool, plus it led to something else I tried, mentioned in the last section)
pd.set_option('display.max_columns', 28)
pd.DataFrame(train_images.iloc[3, :].to_numpy().reshape(28, 28))

In [ ]:
# Some quick data visualization
# (not a terribly exciting dataset)
# First 25 images in training set
# Note: label is above the image

fig, ax = plt.subplots(nrows=5, ncols=5, figsize=(8,8))
fig.subplots_adjust(hspace=.3)
for i in range(5):
    for j in range(5):
        ax[i][j].axis('off')
        ax[i][j].imshow(train_images.iloc[[i+(j*5)], :].to_numpy().astype(np.uint8).reshape(28, 28), cmap='gray')
        ax[i][j].set_title(train_labels[i+(j*5)])

In [ ]:
# Transformations
IMG_SIZE = 28 # size of images in MNIST
# Also the images only have one color channel
# So 3D size = (1, 28, 28)

# Transformations for the train
train_trans = transforms.Compose(([
    transforms.ToPILImage(),
    transforms.RandomCrop(IMG_SIZE), 
    transforms.ToTensor(), # divides by 255
  #  transforms.Normalize((0.5,), (0.5,))
]))

# Transformations for the validation & test sets
val_trans = transforms.Compose(([
    transforms.ToPILImage(),
    transforms.ToTensor(), # divides by 255
   # transforms.Normalize((0.1307,), (0.3081,))
]))

# did worse w/o RandomCrop, if someone can explain why 
# (or if I'm crazy and should try it again) 
# I'd love to hear about it in the comments

In [ ]:
# Custom class for the MNIST dataset from Kaggle
# Images come in a csv, not as actual images
# Training set is split before given to this class

class MNISTDataSet(torch.utils.data.Dataset):
    # images df, labels df, transforms
    # uses labels to determine if it needs to return X & y or just X in __getitem__
    def __init__(self, images, labels, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
                    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        data = self.X.iloc[i, :] # gets the row
        # reshape the row into the image size 
        # (numpy arrays have the color channels dim last)
        data = np.array(data).astype(np.uint8).reshape(28, 28, 1) 
        
        # perform transforms if there are any
        if self.transforms:
            data = self.transforms(data)
        
        # if !test_set return the label as well, otherwise don't
        if self.y is not None: # train/val
            return (data, self.y[i])
        else: # test
            return data

In [ ]:
req_cols = list(test_images.columns)
len(req_cols)

In [ ]:
req_cols.remove("label")
len(req_cols)

In [ ]:
test_images_no_label = test_images[req_cols]
test_images_no_label.shape

In [ ]:
# Get datasets using the custom MNIST Dataset for the train, val, and test images
train_set = MNISTDataSet(train_images, train_labels, train_trans)
val_set = MNISTDataSet(val_images, val_labels, val_trans)
test_set = MNISTDataSet(test_images_no_label, None, val_trans)

# Nice habit to get into
num_classes = 2 # 0-1

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # image starts as (1, 28, 28)
        # Formula to compute size of image after conv/pool
        # (size-filter+2*padding / stride) + 1
        #                      inputs         # of filters    filter size    
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2) # conv1
        self.conv1_bn = nn.BatchNorm2d(num_features=32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2) # conv2
        self.conv2_bn = nn.BatchNorm2d(num_features=64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels= 128, kernel_size=3, stride=1, padding=1) # conv3
        self.conv3_bn = nn.BatchNorm2d(num_features=128)
        
        self.fc1 = nn.Linear(in_features=128*6*6, out_features=1024) # linear 1
        self.fc1_bn = nn.BatchNorm1d(num_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=512) # linear 2
        self.fc2_bn = nn.BatchNorm1d(num_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=256) # linear 3
        self.fc3_bn = nn.BatchNorm1d(num_features=256)
        self.fc4 = nn.Linear(in_features=256, out_features=64) # linear 4
        self.fc4_bn = nn.BatchNorm1d(num_features=64)
        self.out = nn.Linear(in_features=64, out_features=10) # output
    
    def forward(self, t):
        t = F.relu(self.conv1_bn(self.conv1(t)))
        t = F.max_pool2d(t, kernel_size=2, stride=2) # (1, 14, 14)
        
        t = F.relu(self.conv2_bn(self.conv2(t)))
        t = F.max_pool2d(t, kernel_size=2, stride=2) # (1, 7, 7)
        
        t = F.relu(self.conv3_bn(self.conv3(t)))
        t = F.max_pool2d(t, kernel_size=2, stride=1) # (1, 6, 6)
        
        t = F.relu(self.fc1_bn(self.fc1(t.reshape(-1, 128*6*6))))
        t = F.relu(self.fc2_bn(self.fc2(t)))
        t = F.relu(self.fc3_bn(self.fc3(t)))
        t = F.relu(self.fc4_bn(self.fc4(t)))
        t = self.out(t)
        
        return t

In [ ]:
lr = 0.001 # initial learning rate
batch_size = 100 # batch size
epochs = 20 # number of epochs to run

network = Network().to(device) # put the model on device (hopefully a GPU!)
train_dl = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_set, batch_size=batch_size, shuffle=False)
optimizer = optim.Adam(network.parameters(), lr=lr)

for epoch in range(epochs):
    print("Epoch: ", epoch+1)
    epoch_loss = 0
    epoch_correct = 0
    network.train() # training mode
    
    # lessen the learning rate after 4 epochs (0,...,3)
    if epoch == 4:
        print(" decreasing lr")
        optimizer = optim.Adam(network.parameters(), lr=0.00001)
    
    if epoch == 10: # not currently used
        print(" decreasing lr again")
        optimizer = optim.Adam(network.parameters(), lr=0.0000000000001)
    
    for images, labels in train_dl:
        X, y = images.to(device), labels.to(device) # put X & y on device
        y_ = network(X) # get predictions
        
        optimizer.zero_grad() # zeros out the gradients
        loss = F.cross_entropy(y_, y) # computes the loss
        loss.backward() # computes the gradients
        optimizer.step() # updates weights
        
        epoch_loss += loss.item() * batch_size
        epoch_correct += get_num_correct(y_, y)
        
    # Evaluation with the validation set
    network.eval() # eval mode
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for images, labels in val_dl:
            X, y = images.to(device), labels.to(device) # to device
            
            preds = network(X) # get predictions
            loss = F.cross_entropy(preds, y) # calculate the loss
            
            val_correct += get_num_correct(preds, y)
            val_loss = loss.item() * batch_size
    # Print the loss and accuracy for the validation set
    print(" Val Loss: ", val_loss)
    print(" Val Acc: ", (val_correct/len(val_images))*100)

In [ ]:
# Use the validation set to make a confusion matrix
network.eval() # good habit I suppose
predictions = torch.LongTensor().to(device) # Tensor for all predictions

# Goes through the val set
# Don't care about the associated labels
for images, _ in val_dl:
    preds = network(images.to(device))
    predictions = torch.cat((predictions, preds.argmax(dim=1)), dim=0)

predictions

In [ ]:
#predictions = [x if not x else 1 for x in predictions]
# Make the confusion matrix
cmt = torch.zeros(num_classes, num_classes, dtype=torch.int32)
for i in range(len(val_labels)):
    cmt[val_labels[i], predictions[i]] += 1


In [ ]:
predictions

In [ ]:
cmt

In [ ]:
# Time to get the network's predictions on the test set
# Put the test set in a DataLoader
test_dl = DataLoader(test_set, batch_size=batch_size, shuffle=False)

network.eval() # Safety first
predictions = torch.LongTensor().to(device) # Tensor for all predictions

# Go through the test set, saving the predictions in... 'predictions'
for images in test_dl:
    preds = network(images.to(device))
    predictions = torch.cat((predictions, preds.argmax(dim=1)), dim=0)

In [ ]:
y_true = test_images['label']
y_pred = predictions.cpu().numpy()

len(y_true), len(y_pred)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_true, y_pred))